# Interactive ORPO Fine-Tuning & Inference Hub for Open LLMs

This experiment provides an interactive and modular interface for selecting, downloading, fine-tuning, and evaluating large language models using ORPO (Optimal Reward Preferring Optimization).
The user can choose between state-of-the-art open LLMs like Mistral, LLaMA 2/3, and Gemma. 

# Overview

## 📦 Imports

By using our Local GenAI workspace image, most of the necessary libraries to work with ORPO-based fine-tuning and evaluation already come pre-installed. In this notebook, we only need to import components for model loading, quantization, inference, and feedback visualization to run the complete ORPO workflow locally

In [11]:
!pip install -r ../requirements.txt --quiet

In [12]:
import os
import sys
import yaml
from pathlib import Path


# Define the relative path to the 'src' directory (two levels up from current working directory)
src_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Add 'src' directory to system path for module imports (e.g., utils)
if src_path not in sys.path:
    sys.path.append(src_path)

In [13]:
# ===============================
# 🧠 Core Libraries
# ===============================
import torch
import multiprocessing
import mlflow
from datasets import load_dataset

# ===============================
# 🧪 Hugging Face & Transformers
# ===============================
from huggingface_hub import login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)

# ===============================
# 🧩 Fine-tuning (ORPO + PEFT)
# ===============================
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

# ===============================
# 🧰 Project Modules: Core Pipeline
# ===============================
from core.selection.model_selection import ModelSelector
from core.local_inference.inference import InferenceRunner
from core.target_mapper.lora_target_mapper import LoRATargetMapper
from core.data_visualizer.feedback_visualizer import UltraFeedbackVisualizer
from core.finetuning_inference.inference_runner import AcceleratedInferenceRunner
from core.merge_model.merge_lora import merge_lora_and_save
from core.quantization.quantization_config import QuantizationSelector

# ===============================
# 🚀 Deployment & Evaluation
# ===============================
from core.deploy.deploy_fine_tuning import register_llm_comparison_model
from core.comparer.galileo_hf_model_comparer import GalileoLocalComparer
import promptquality as pq

# ===============================
# ⚙️ Utility Functions
# ===============================
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    setup_galileo_environment,
    initialize_galileo_evaluator,
    initialize_galileo_protect,
    initialize_galileo_observer,
    login_huggingface
)


## Configurations

In [14]:
CONFIG_PATH = "../../configs/config.yaml"
SECRETS_PATH = "../../configs/secrets.yaml"
GALILEO_EVALUATE_PROJECT_NAME="AIStudio-Fine-Tuning-Evaluate"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Fine-Tuning-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Fine-Tuning-Run"
MLFLOW_MODEL_NAME = "AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_RUN_NAME="AIStudio-Fine-Tuning-Service-Run"
MODEL_SERVICE_NAME="AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_EXPERIMENT_NAME="AIStudio-Fine-Tuning-Experiment"


### Proxy Configuration
In order to connect to Galileo service, a SSH connection needs to be established. For certain enterprise networks, this might require an explicit setup of the proxy configuration. If this is your case, set up the "proxy" field on your config.yaml and the following cell will configure the necessary environment variable.

In [15]:
configure_proxy(CONFIG_PATH)

### 🔍 Model Selector

Below are the available models for fine-tuning with ORPO.  
> ⚠️ **Note:** Make sure your Hugging Face account has access permissions for the selected model (some require manual approval).

| Model ID | Hugging Face Link |
|----------|-------------------|
| `mistralai/Mistral-7B-Instruct-v0.1` | [🔗 View on Hugging Face](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) |
| `meta-llama/Llama-2-7b-chat-hf` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) |
| `meta-llama/Meta-Llama-3-8B-Instruct` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) |
| `google/gemma-7b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-7b-it) |
| `google/gemma-3-1b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-3-1b-it) |


In [16]:
MODEL =  "google/gemma-3-1b-it"

### 🔐 Login to Hugging Face

To access gated models (e.g., LLaMA, Mistral, or Gemma), you must authenticate using your Hugging Face token.

Make sure your `secrets.yaml` file contains the following key:

```yaml
HUGGINGFACE_API_KEY: your_huggingface_token

In [17]:
config, secrets = load_config_and_secrets()
login_huggingface(secrets)

✅ Logged into Hugging Face successfully.


### Attention Optimization Config
Automatically selects the most efficient attention implementation and data type (dtype) based on the GPU’s compute capability.

In [7]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

## Model Loader

In [18]:
selector = ModelSelector()
selector.select_model(MODEL)

model = selector.get_model()
tokenizer = selector.get_tokenizer()


INFO:ModelSelector:[ModelSelector] Selected model: google/gemma-3-1b-it
INFO:ModelSelector:[ModelSelector] Downloading model snapshot to: ../../../local/models/google__gemma-3-1b-it


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:ModelSelector:[ModelSelector] ✅ Model downloaded successfully to: ../../../local/models/google__gemma-3-1b-it
INFO:ModelSelector:[ModelSelector] Loading model and tokenizer from: ../../../local/models/google__gemma-3-1b-it
INFO:ModelSelector:[ModelSelector] Checking model for ORPO compatibility...
INFO:ModelSelector:[ModelSelector] ✅ Model 'google/gemma-3-1b-it' is ORPO-compatible.


## 🤖 Inference with Default Model

The following cell runs inference using the base (non fine-tuned) model you selected earlier.

We've prepared a few prompts to test different types of reasoning and writing skills.  
You can later compare these outputs with the results generated by the fine-tuned model.

In [10]:
# 📋 Custom prompts for evaluation
prompts = [
    "I need to write some nodejs code that publishes a message to a Telegram group.",
    "What advice would you give to a frontend developer?",
    "Propose a solution that could reduce the rate of deforestation.",
    "Write a eulogy for a public figure who inspired you."
]

# ⚙️ Run inference with the selected model
runner = InferenceRunner(selector)

for idx, prompt in enumerate(prompts, 1):
    response = runner.infer(prompt)
    print(f"\n🟢 Prompt {idx}: {prompt}\n🔽 Model Response:\n{response}\n{'-'*80}")


INFO:InferenceRunner:[InferenceRunner] Detected 2 GPUs, loading multi-GPU configuration.
INFO:InferenceRunner:[InferenceRunner] Loading model and tokenizer from snapshot at: ../../../local/models/google__gemma-3-1b-it
INFO:InferenceRunner:[InferenceRunner] Running inference on input: I need to write some nodejs code that publishes a message to a Telegram group....
INFO:InferenceRunner:[InferenceRunner] Inference completed.
INFO:InferenceRunner:[InferenceRunner] Running inference on input: What advice would you give to a frontend developer?...



🟢 Prompt 1: I need to write some nodejs code that publishes a message to a Telegram group.
🔽 Model Response:
I need to write some nodejs code that publishes a message to a Telegram group.

Here's the code:

```javascript
const Telegram = require('node-telegram-bot-api');

// Replace with your Telegram bot token
const token = 'YOUR_BOT_TOKEN';

// Create a new bot instance
const bot = new Telegram.Bot(token);

// Replace with your Telegram group ID
const groupId = 'YOUR_GROUP_ID';

// Send the message
bot.sendMessage(groupId, 'Hello, this is a
--------------------------------------------------------------------------------


INFO:InferenceRunner:[InferenceRunner] Inference completed.
INFO:InferenceRunner:[InferenceRunner] Running inference on input: Propose a solution that could reduce the rate of deforestation....



🟢 Prompt 2: What advice would you give to a frontend developer?
🔽 Model Response:
What advice would you give to a frontend developer?

Okay, here's a breakdown of advice I'd give a frontend developer, categorized for clarity:

**1. Foundational Skills - The Bedrock:**

* **Master the Basics:**  HTML, CSS, and JavaScript are *everything*. Don't skip them. Understand the fundamentals: DOM manipulation, event handling, basic CSS selectors, and how JavaScript works.
* **Version Control (Git):**  This is non-negotiable. Learn to use Git and GitHub
--------------------------------------------------------------------------------


INFO:InferenceRunner:[InferenceRunner] Inference completed.
INFO:InferenceRunner:[InferenceRunner] Running inference on input: Write a eulogy for a public figure who inspired you....



🟢 Prompt 3: Propose a solution that could reduce the rate of deforestation.
🔽 Model Response:
Propose a solution that could reduce the rate of deforestation.

**Proposed Solution: Integrated Agroforestry with Community-Based Monitoring and Incentives**

This solution combines several approaches to tackle deforestation, focusing on sustainable land management, community engagement, and economic benefits.

**1. Integrated Agroforestry (IA):**

* **What it is:** IA involves integrating trees and shrubs into agricultural systems. This is not just planting trees; it’s creating a productive ecosystem that provides multiple benefits – food, timber, fuelwood, soil stabilization, and carbon sequestration.
--------------------------------------------------------------------------------


INFO:InferenceRunner:[InferenceRunner] Inference completed.



🟢 Prompt 4: Write a eulogy for a public figure who inspired you.
🔽 Model Response:
Write a eulogy for a public figure who inspired you.

---

The air feels… quiet. A quiet that isn’t peaceful, but a quiet born of absence. We’re here today to say goodbye to [Name], a voice that resonated with so much passion, so much hope, and so much unwavering belief in the power of human connection.

[Name] wasn’t a politician, a celebrity, or a titan of industry. They were, simply, [Describe their core identity - e.g., a teacher, a community
--------------------------------------------------------------------------------


## 🏷️ Creating the Fine-Tuned Model Name (ORPO)

We define a clean and consistent name for the fine-tuned version of the selected base model

In [19]:
base_model = selector.model_id
model_path = selector.format_model_path(base_model)
new_model = f"Orpo-{base_model.split('/')[-1]}-FT"
fine_tuned_name = f"Orpo-{base_model.split('/')[-1]}-FT"
fine_tuned_path = f"../../../local/models_llora/{fine_tuned_name}"

### ⚙️  Automatic Quantization Configuration

We use an intelligent selector to automatically choose the optimal quantization strategy for the hardware environment.

- `QuantizationSelector()` analyzes the number of available GPUs and their memory.
- If multiple GPUs with sufficient VRAM are detected, it applies 8-bit quantization for faster performance.
- Otherwise, it falls back to `4-bit QLoRA` using `nf4` and double quantization to reduce memory usage.

This adaptive configuration ensures efficient fine-tuning of large language models by balancing performance and hardware constraints.

In [13]:
quantization = QuantizationSelector()
bnb_config = quantization.get_config()

✅ Using 8-bit quantization (sufficient GPUs and VRAM available).


### 🧩 PEFT Configuration (LoRA)

We define the LoRA configuration using the `LoraConfig` from PEFT (Parameter-Efficient Fine-Tuning).


In [20]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LoRATargetMapper.get_target_modules(base_model)
)

### 🧠 Load and Prepare Base Model for Training

In this step, we load the base model and tokenizer from the local path, apply the quantization configuration (`bnb_config`), prepare it for tra

In [21]:
model_vocab_size = AutoModelForCausalLM.from_pretrained(model_path).config.vocab_size
tokenizer_vocab_size = len(tokenizer)

if tokenizer_vocab_size != model_vocab_size:
    print(f"⚠️ Adjusting vocabulary ({tokenizer_vocab_size}) ≠ Model ({model_vocab_size})")
    tokenizer.pad_token = tokenizer.eos_token  
    tokenizer.save_pretrained(model_path)

⚠️ Adjusting vocabulary (262145) ≠ Model (262144)


In [22]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map={"": 0},
)

In [23]:
# Safely apply chat format only if tokenizer doesn't already have a chat_template
if tokenizer.chat_template is None:
    model, tokenizer = setup_chat_format(model, tokenizer)
else:
    print("⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.")


⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.


In [24]:
model = prepare_model_for_kbit_training(model)


## 📚 Dataset Loader

We use the [UltraFeedback Binarized](https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized) dataset provided by Hugging Face.

This dataset contains prompts along with two model-generated responses:
- **chosen**: the response preferred by human annotators
- **rejected**: the less preferred one

For this experiment, we load a subset of the data to speed up training and evaluation.  
A fixed seed ensures reproducibility when shuffling the data.


In [25]:
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# 📊 Define sample sizes for a lightweight experiment
train_samples = 5000                         # Subset size for training
original_train_samples = 61135              # Total training examples in the original dataset
test_samples = int((2000 / original_train_samples) * train_samples)  # Proportional test size

# 🔀 Shuffle and sample subsets from both splits
train_subset = dataset[0].shuffle(seed=42).select(range(train_samples))
test_subset = dataset[1].shuffle(seed=42).select(range(test_samples))


### 📊 Dataset Visualization

To help understand how the dataset works, we use the `UltraFeedbackVisualizer`.

This tool logs examples from the dataset into **TensorBoard**, including:
- The **original prompt** given to the model
- The two possible answers: one **preferred by humans** and one that was **rejected**
- A simple comparison showing which response was rated better

Each example is displayed with clear labels and scores to help illustrate the kinds of outputs humans value more — **before we do any fine-tuning**.

> This is useful to explore what “good answers” look like, based on real human feedback.


In [26]:
visualizer = UltraFeedbackVisualizer(train_subset, test_subset,max_samples=20)
visualizer.run()

2025-04-23 12:44:52,048 - INFO - Use pytorch device_name: cuda
2025-04-23 12:44:52,050 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-04-23 12:44:55,882 - INFO - 📊 Logging training samples (human feedback only)...
2025-04-23 12:44:55,991 - INFO - [Example 0] ✅ Logged successfully
2025-04-23 12:44:56,023 - INFO - [Example 1] ✅ Logged successfully
2025-04-23 12:44:56,061 - INFO - [Example 2] ✅ Logged successfully
2025-04-23 12:44:56,095 - INFO - [Example 3] ✅ Logged successfully
2025-04-23 12:44:56,132 - INFO - [Example 4] ✅ Logged successfully
2025-04-23 12:44:56,169 - INFO - [Example 5] ✅ Logged successfully
2025-04-23 12:44:56,206 - INFO - [Example 6] ✅ Logged successfully
2025-04-23 12:44:56,293 - INFO - [Example 7] ✅ Logged successfully
2025-04-23 12:44:56,345 - INFO - [Example 8] ✅ Logged successfully
2025-04-23 12:44:56,374 - INFO - [Example 9] ✅ Logged successfully
2025-04-23 12:44:56,419 - INFO - [Example 10] ✅ Logged successfully
2025-04-23 12:44:56,474 - I

In [27]:
def process(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset[0] = train_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

dataset[1] = test_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(dataset)

Map (num_proc=48):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=48):   0%|          | 0/163 [00:00<?, ? examples/s]

[Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 5000
}), Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 163
})]


## ⚙️ ORPO Configuration

We define the training configuration using the `ORPOConfig` class from TRL (Transformers Reinforcement Learning).

This configuration controls how the model will be fine-tuned using ORPO (Offline Reinforcement Preference Optimization), a technique that aligns model outputs with human preferences.

Key parameters include:
- `learning_rate`: sets how fast the model updates (8e-6 is typical for PEFT)
- `beta`: the strength of the ORPO loss term
- `optim`: uses 8-bit optimizer for memory efficiency (paged_adamw_8bit)
- `max_steps`: controls how long training will run (e.g., 1000 steps)
- `eval_strategy` and `eval_steps`: defines how and when to evaluate during training
- `output_dir`: directory to save the trained model

> This configuration is compatible with all the selected models (e.g., Mistral, LLaMA, Gemma) and optimized for QLoRA fine-tuning on consumer or research-grade GPUs.


In [ ]:
mlflow.set_tracking_uri('/phoenix/mlflow')
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    max_steps=1000,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to=["mlflow","tensorboard"],
    output_dir="./results/",
)

2025/04/23 12:45:11 INFO mlflow.tracking.fluent: Experiment with name 'AIStudio-Fine-Tuning-Experiment' does not exist. Creating a new experiment.


### 🚀 ORPO Trainer

We now initialize the `ORPOTrainer`, which orchestrates the fine-tuning process using the Offline Reinforcement Preference Optimization (ORPO) strategy.

It takes as input:
- The **base model**, already prepared with QLoRA and chat formatting
- The **ORPO configuration** (`orpo_args`) containing all training hyperparameters
- The **training and evaluation datasets**
- The **LoRA configuration** (`peft_config`) for parameter-efficient fine-tuning
- The **tokenizer**, passed as a `processing_class`, to apply proper formatting and padding

Once initialized, the trainer will be ready to start training with `trainer.train()`.


In [29]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset[0],
    eval_dataset=dataset[1],
    peft_config=peft_config,
    processing_class=tokenizer  
)

/opt/conda/lib/python3.12/site-packages/trl/trainer/orpo_trainer.py:275: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()
trainer.save_model(fine_tuned_path)

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
/opt/conda/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


In [4]:
merge_lora_and_save(
    base_model_id=MODEL,
    finetuned_lora_path=fine_tuned_path
)


🧹 Cleaning up memory...
🔄 Loading tokenizer and base model...


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


⚠️ Resizing token embeddings: model (262144) → tokenizer (262145)
⚠️ Tokenizer already has chat_template. Skipping setup_chat_format.
🔗 Loading LoRA weights from: ./Orpo-gemma-3-1b-it-FT
🧠 Merging LoRA weights...
💾 Saving merged model to: ../../../local/models_llora/Orpo-gemma-3-1b-it-FT
✅ Finished! Model successfully merged and saved locally.


In [14]:
fine_tuned_path = f"../../../local/models_llora/{fine_tuned_name}"

tokenizer = AutoTokenizer.from_pretrained(fine_tuned_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_path, torch_dtype=torch.float16).cuda().eval()

prompt = "Propose a solution that could reduce the rate of deforestation"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=500)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Propose a solution that could reduce the rate of deforestation, including at least 12 different examples that are specific to locations such as Amazonia, and at least 2 in 20 different regions worldwide, one of which is the European Union.user
Propose a solution that could reduce the rate of deforestation, including at least 12 different examples that are specific to locations such as Amazonia, and at least 2 in 20 different regions worldwide, one of which is the European Union.


## Galileo Evaluate
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo Evaluate to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

Galileo Evaluate is a platform designed to optimize and simplify the experimentation and evaluation of generative AI systems, especially large language model (LLM) applications. Its goal is to facilitate the process of building AI systems with deep insights and collaborative tools, replacing fragmented experimentation in spreadsheets and notebooks with a more integrated approach.

You can log metrics in Galileo Evaluate and track all your experiments in one place. In our example, we logged several questions, selected specific metrics, and ran a batch of experiments to evaluate our chain. To learn more about the available metrics, see: Galileo Guardrail Metrics.

In [15]:
#########################################
# In order to connect to Galileo, create a secrets.yaml file in the configs folder.
# This file should be an entry called GALILEO_API_KEY, with your personal Galileo API Key
# Galileo API keys can be created on https://console.hp.galileocloud.io/settings/api-keys
#########################################

setup_galileo_environment(secrets)
pq.login(os.environ['GALILEO_CONSOLE_URL'])

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as diogo.vieira@hp.com.


Config(console_url=HttpUrl('https://console.hp.galileocloud.io/'), username=None, password=None, api_key=SecretStr('**********'), token=SecretStr('**********'), current_user='diogo.vieira@hp.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=HttpUrl('https://api.hp.galileocloud.io/'))

In [10]:
comparer = GalileoLocalComparer(
    base_selector=selector,
    finetuned_path=fine_tuned_path,
    prompts=[
        "Explain the importance of sustainable agriculture.",
        "Write a Python function to check for palindromes.",
    ],
    galileo_project_name=GALILEO_EVALUATE_PROJECT_NAME,
    dtype=torch.float16
)

comparer.compare()

INFO:AcceleratedInferenceRunner:🔄 Loading tokenizer and base model from ModelSelector...
INFO:AcceleratedInferenceRunner:✅ Model loaded and ready for inference.
INFO:AcceleratedInferenceRunner:🔄 Loading tokenizer and base model from ModelSelector...
INFO:AcceleratedInferenceRunner:🎯 Applying LoRA fine-tuned weights...
INFO:AcceleratedInferenceRunner:✅ Model loaded and ready for inference.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Explain the importance of sustainable agriculture....


⚙️ Running prompt 1/2


INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Explain the importance of sustainable agriculture....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Write a Python function to check for palindromes....


⚙️ Running prompt 2/2


INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:AcceleratedInferenceRunner:🔍 Running inference for prompt (truncated): Write a Python function to check for palindromes....
INFO:AcceleratedInferenceRunner:✅ Inference complete.
INFO:promptquality.utils.logger:Project AIStudio-Fine-Tuning-Evaluate already exists, using it.


Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rag_nli: Done ✅
instruction_adherence: Computing 🚧
cost: Done ✅
toxicity: Done ✅
pii: Done ✅
protect_status: Done ✅
latency: Done ✅
factuality: Computing 🚧
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/48f07fac-cf49-4fb0-ba74-c4c109edd1e4/550cfb8a-70e6-468f-baf2-93e1b7f4b265?taskType=12
✅ Finished logging outputs for both models to Galileo.


Built with ❤️ using Z by HP AI Studio.